In [1]:
import numpy as np
import tensorflow as tf
import os
from scipy.special import softmax

In [2]:
relu = lambda x: x * (x > 0)

In [3]:
# Combine test and train images together into one dataset

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0  

all_images =np.concatenate([train_images, test_images], axis=0)
all_labels =np.concatenate([train_labels, test_labels], axis=0)

In [4]:
# Arguments

layer_width = 512
model_seed = 52233264

In [5]:
# load weights
tag = "mnist_dense" + '-w' + str(layer_width) + 'x' + str(layer_width) + '-' + str(model_seed)
cur_folder = os.getcwd()
weights_folder = os.path.join(cur_folder,"weights",tag)
weight_filename_tag = os.path.join(weights_folder, tag + "_")

# Layer_0
dense_weights_file = weight_filename_tag + "dense_weights.npy"
dense_biases_file = weight_filename_tag + "dense_biases.npy"
dense_weights = np.load(dense_weights_file)
dense_biases = np.load(dense_biases_file).reshape(-1,1)

# Layer_1
dense_1_weights_file = weight_filename_tag + "dense_1_weights.npy"
dense_1_biases_file = weight_filename_tag + "dense_1_biases.npy"
dense_1_weights = np.load(dense_1_weights_file)
dense_1_biases = np.load(dense_1_biases_file).reshape(-1,1)

#Layer_2
dense_2_weights_file = weight_filename_tag + "dense_2_weights.npy"
dense_2_biases_file = weight_filename_tag + "dense_2_biases.npy"
dense_2_weights = np.load(dense_2_weights_file)
dense_2_biases = np.load(dense_2_biases_file).reshape(-1,1)

In [6]:
# load input
image_no = 69999;
input_image = all_images[image_no]
input_image_label = all_labels[image_no]
print("IMAGE_NUMBER: ", image_no)
print("TRUTH_LABEL: ", input_image_label)

IMAGE_NUMBER:  69999
TRUTH_LABEL:  6


In [7]:
# flatten input

flattened_input = input_image.reshape(-1,1)

In [8]:
# send through Layer_0

# matvec multiplication
l0 = np.matmul(dense_weights.transpose(), 
               flattened_input)
# add bias
b0 = l0 + dense_biases

# ReLU
b0_relu = relu(b0)

In [ ]:
# def block_matvec_mul(mat_A, vec_B, BLOCK_WIDTH, BLOCK_HEIGHT):
#     gridydim = np.ceil(mat_A.shape[0]/BLOCK_HEIGHT)
#     gridxdim = np.ceil(mat_A.shape[1]/BLOCK_WIDTH)

In [14]:
mat_A = dense_weights.transpose()
vec_B = flattened_input
BLOCK_WIDTH = 98 
BLOCK_HEIGHT = 64
gridydim = int(np.ceil(mat_A.shape[0]/BLOCK_HEIGHT))
gridxdim = int(np.ceil(mat_A.shape[1]/BLOCK_WIDTH))

In [25]:
block_result = np.zeros((gridxdim, gridydim,BLOCK_HEIGHT ))
for x in range(gridxdim):
    for y in range(gridydim):
        block_result[x][y] = np.matmul(mat_A[y*BLOCK_HEIGHT:(y+1)*BLOCK_HEIGHT,
                                             x*BLOCK_WIDTH:(x+1)*BLOCK_WIDTH], vec_B[x*BLOCK_WIDTH:(x+1)*BLOCK_WIDTH]).squeeze()

In [30]:
block_result[0][0]

array([-0.2271778 , -0.05175527,  0.05430012, -0.07369927,  0.03136926,
        0.01070307, -0.00880489, -0.17959148,  0.01069862,  0.09602164,
        0.30006996, -0.07533787,  0.08697767, -0.11873722,  0.04214808,
       -0.09910676,  0.07967037, -0.02747371, -0.10535175, -0.07890169,
       -0.23723087, -0.00216037,  0.02920011, -0.06236065, -0.02466929,
        0.09762336,  0.14536835, -0.08125498,  0.03984001, -0.04781194,
       -0.02764399,  0.08038249,  0.13794455, -0.04274514, -0.01352616,
       -0.22639287,  0.00608538, -0.01955259, -0.01390323, -0.29229653,
        0.14870457,  0.11163494,  0.08664815, -0.06725822, -0.11501683,
       -0.04009764, -0.04251356,  0.06426201,  0.10306149,  0.19338332,
        0.1153013 ,  0.05630733,  0.17276248, -0.06732856,  0.02955971,
        0.16069503,  0.06314635, -0.06719813, -0.24348572, -0.20874313,
       -0.007449  ,  0.10988941,  0.03196   ,  0.08673925])

In [31]:
error_profile = np.random.uniform(low=0.0, high=0.6, size=(gridxdim, gridydim,BLOCK_HEIGHT))

In [9]:
# send through Layer_1

# matvec multiplication
l1 = np.matmul(dense_1_weights.transpose(), 
               b0_relu)
# add bias
b1 = l1 + dense_1_biases

# ReLU
b1_relu = relu(b1)

In [10]:
# send through Layer_2

# matvec multiplication
l2 = np.matmul(dense_2_weights.transpose(), 
               b1_relu)
# add bias
b2 = l2 + dense_2_biases

# softmax
b2_softmax = softmax(b2)

In [11]:
# Output
prediction = np.argmax(b2_softmax)
print("PREDICTION: ", prediction)

PREDICTION:  6
